In [1]:
import tellurium as te

In [2]:
class Gene:
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return self.name

In [3]:
class Circuit:
    def __init__(self, *components):
        self.circuit = "\n".join(components)

    def __str__(self):
        return self.circuit

def circuit_degrade(x: Gene, k: float):
    return f"{x} -> ; {k} * {x}"

def circuit_and(x: Gene, y: Gene, z: Gene, nx: float=2, ny: float=2):
    return f" -> {z}; {x}^{nx} * {y}^{ny} / (1 + {x}^{nx}) / (1 + {y}^{ny})"

x = Gene("xa")
y = Gene("ya")
z = Gene("za")

circuit = Circuit(
    circuit_degrade(x, 2),
    circuit_and(x, y, z, 2, 2),
)

In [8]:
from typing import Any


class Cell:
    def __init__(self, name, circuit):
        self.name = name
        self.genes = {}
        self._circuit = circuit

    def update_genes(self, result):
        for col in result.colnames[1:]:
            gene_name = col[1:-1]
            gene_level = result[col][-1]
            self[gene_name] = gene_level
    
    def __setitem__(self, k, v):
        self.genes[str(k)] = v

    def __contains__(self, k):
        return str(k) in self.genes()

    def __getitem__(self, k):
        return self.genes[str(k)]
    
    @property
    def circuit(self):
        prefix = f"model {self.name}"
        suffix = "end"
        gene_levels = "\n".join([f"{gene} = {level}" for gene, level in self.genes.items()])
        return "\n".join([prefix, str(self._circuit), gene_levels, suffix])
        

In [5]:
cell1 = Cell("cell1", circuit)
cell1[x] = 1
cell1[y] = 0.5
cell1[z] = 0

print(cell1.circuit)

model cell1
xa -> ; 2 * xa
 -> za; xa^2 * ya^2 / (1 + xa^2) / (1 + ya^2)
xa = 1
ya = 0.5
za = 0
end


In [6]:
rr = te.loada(cell1.circuit)
result = rr.simulate(0, 1, 5)
result.colnames

['time', '[xa]', '[za]']

In [10]:
cell1.genes

{<__main__.Gene object at 0x14f4c2aa0>: 1, <__main__.Gene object at 0x14f4c3820>: 0.5, <__main__.Gene object at 0x14f4c1600>: 0}
